# Hotspot calculation demo


This notebook detects and classifies hotspots of the von Mises stress in a connected FEM mesh. Each element/node entry of the mesh receives a number of the hotspot it is member of. "0" means the element/node is not part of any hotspots. "1" means that the element/node is part of the hotspot with the highes peak, "2" the same for the second highest peak and so forth.

See [module documentation](https://pylife.readthedocs.io/en/latest/mesh/hotspot.html) further details.

In [ ]:
import numpy as np
import pylife
import pandas as pd
import scipy.stats as stats
import pylife.stress.equistress
import pylife.strength.meanstress
import pylife.mesh.meshsignal
import pylife.mesh.hotspot
import pylife.vmap

import pyvista as pv

pv.set_plot_theme('document')
pv.set_jupyter_backend('panel')

In [ ]:
vm_mesh = pylife.vmap.VMAPImport("plate_with_hole.vmap")
pyLife_mesh = (vm_mesh.make_mesh('1', 'STATE-2')
               .join_coordinates()
               .join_variable('STRESS_CAUCHY')
               .join_variable('DISPLACEMENT')
               .to_frame())

## Equivalent stress calculation

In [ ]:
pyLife_mesh['mises'] = pyLife_mesh.equistress.mises()

## Hot spot Calculation ##

In [ ]:
threshold = .9 # factor of the maximum local value
pyLife_mesh['hotspot'] = pyLife_mesh.hotspot.calc("mises", threshold)
display(pyLife_mesh[['x', 'y', 'z', 'mises', 'hotspot']])

In [ ]:
print("%d hotspots found over the threshold" % pyLife_mesh['hotspot'].max())

In [ ]:
grid = pv.UnstructuredGrid(*pyLife_mesh.mesh.vtk_data())
plotter = pv.Plotter(window_size=[1920, 1080])
plotter.add_mesh(grid, scalars=pyLife_mesh.groupby('element_id')['hotspot'].first().to_numpy(),
                show_edges=True, cmap='prism_r')
plotter.add_scalar_bar()
plotter.show()

### First hotspot

In [ ]:
display(pyLife_mesh[pyLife_mesh['hotspot'] == 1])